# 硫酸製造機のスループット最大時の最小構成を求める

一次の混合整数計画問題なのでPuLPで解ける  
硫黄は無限にあるとする

Author: hagino3000  
Date: 2017-12-01

In [5]:
import pulp

In [33]:
def solve(h2o, machine1_speed, machine2_speed, machine3_speed):
    """
    Parameters
    ==========
    h2o: int
        水の供給量/sec
    machine1_speed: int
        純水→水素＋酸素の処理能力
        H2の生成速度 = machine1_speed/sec
        O2の生成速度 = 0.5*machine1_speed/sec
    machine2_speed: int
        酸素＋硫黄→二酸化硫黄の処理能力
        SO2の生成速度 = machine2_speed/sec
    machine3_speed
        二酸化硫黄＋純水→硫酸の処理能力
        H2SO4の生成速度 = machine3_speed/sec
    
    """
    problem = pulp.LpProblem(sense=pulp.LpMaximize)
    # 変数
    x = pulp.LpVariable('Number of Machine1 純水→水素＋酸素', cat=pulp.LpInteger, lowBound=0)
    y = pulp.LpVariable('Number of Machine2 酸素＋硫黄→二酸化硫黄', cat=pulp.LpInteger, lowBound=0)
    z = pulp.LpVariable('Number of Machine3 二酸化硫黄＋純水→硫酸', cat=pulp.LpInteger, lowBound=0)
    r = pulp.LpVariable('H2O Rate to create O2', cat=pulp.LpVariable, lowBound=0, upBound=1)
    
    h2 = h2o * r
    o2 = h2o * r * 0.5
    so2 = o2
    h2so4 = so2
    
    # 問題
    problem += h2so4 - (x + y + z)
    
    # 制約 マシンの製造能力以上作れない
    problem += h2 <= x * machine1_speed
    problem += o2 <= x * machine1_speed * 0.5     
    problem += so2 <= y * machine2_speed
    problem += h2so4 <= z * machine3_speed
    # 制約 硫酸１作るのに純水１必要
    problem += h2so4 <= h2o * (1 - r)
    
    status = problem.solve()
    print(pulp.LpStatus[status])
    return x.value(), y.value(), z.value(), r.value()

In [30]:
solve(300, 100, 10, 5)

Optimal


(2.0, 10.0, 20.0, 0.66666667)

In [31]:
solve(300, 10, 10, 10)

Optimal


(20.0, 10.0, 10.0, 0.66666667)

In [32]:
solve(300, 10, 20, 10)

Optimal


(20.0, 5.0, 10.0, 0.66666667)